---

# CSCI 3352, Spring 2022
# FINAL PROJECT

<br> 

### Owen Smith, Kyle Ma

<br> 

In [25]:
import os
import random
import numpy as np
import pandas as pd
import networkx as nx
import math
import matplotlib
import matplotlib.pylab as plt
from sklearn import tree
from pyvis.network import Network
import csv

In [26]:
### ITERATE THROUGH NETWORKS DIRECTORY, HANDLE FILES ###
directory = 'networks'
for filename in os.scandir(directory):
    filename = filename.name
    name, ext = os.path.splitext(filename)
    ext = str(ext)[0:4]
    if ext == '.csv' and filename != 'references.csv':
        data = np.loadtxt(open(directory + '/' + filename, "rb"), delimiter=",", skiprows=1)
        edgelist = ''
        listfile = open(directory + '/' + filename[:-4] + '.txt', 'w')
        for row in range(0, len(data)):
            for col in range(0, len(data[row])):
                if data[row][col] != 0:
                    edgelist = edgelist + str(col) + ' ' + str(row) + '\n'
        d = listfile.write(edgelist)
        listfile.close()

In [27]:
# takes in a graph G and returns a list of network statistics
def gather_network_stats(G):
    def compute_MGD(G):
        path = list((dict(nx.all_pairs_shortest_path_length(G)).values())) #takes return and converts to list of dicts
        Z = 0
        l_sum = 0

        for node in path:
            tmp = list(node.values()) #convert node dict to list of values
            tmp.remove(0) #remove all instances of 0
            Z = Z + len(tmp) #calculate Z from size of list
            l_sum = l_sum + sum(tmp) #add all the path lengths to the total for the numerator later on

        MGD = l_sum / Z
        return MGD
    
    def percent_single_degree(G): #gets percentage of nodes with degree of one, as these edge suggest specialized predation
        degrees = dict(G.degree())
        degrees = list(degrees.values())
        single_degree_count = 0
        for k in degrees:
            if k == 1 or k == 2:
                single_degree_count += 1
        return single_degree_count / G.number_of_nodes()

    def count_FFBL_motifs(G,flag):
        FFL_count = 0
        FBL_count = 0

        ### MY CODE ###
        FFL_pairs = [] #lists that will contain pairs for later printing if flag == 1
        FBL_pairs = []

        for i in G.nodes(): #loop for suggested enumeration
            for j in G.neighbors(i): #paths of length 1
                for k in G.neighbors(j): #paths of length 2
                    if (i != k): #eliminate bidirectionals
                        if (G.has_edge(i, k)): #if there is an edge from i -> k, it is a FFL
                            FFL_count += 1
                            FFL_pairs.append(((i,j), (j,k), (i,k)))
                        if (G.has_edge(k, i)): #if there is an edge from k -> i, it is a FBL
                            FBL_count += 1
                            FBL_pairs.append(((i,j), (j,k), (k,i)))

        FBL_count = int(FBL_count / 3) #divide FBL_count by 3 to retroactively account for duplicates

        if (flag == 1): #print flag
            for i in FFL_pairs:
                print("FFL: ", i)
            FBL_pairs = FBL_pairs[::3] #takes every third element to get rid of duplicates (kinda jank but it works)
            for i in FBL_pairs:
                print("FBL:", i)

        return (FFL_count,FBL_count)
    
    n = G.number_of_nodes() #builtin function that returns number of nodes as an int
    m = G.number_of_edges() #builtin funciton that reutnrs number of edges as an int
    
    degrees = dict(G.degree())
    totalDegrees = sum(degrees.values()) #get values, sum, divide by number of nodes
    kmean =  totalDegrees / n #calculate mean
    
    degree_lst = list(degrees.values())
    kmax = max(degree_lst)
    
    C = nx.transitivity(G)
    node_connectivity = nx.node_connectivity(G)
    MGD = compute_MGD(G)
    FFL_count, FBL_count = count_FFBL_motifs(G, 0)
    FFL_count = FFL_count / n
    FBL_count = FBL_count / n
    specialized_nodes = percent_single_degree(G)
    
    percent_cannibal = 0
    for node in G.nodes():
        if node in G.neighbors(node):
            percent_cannibal += 1
    percent_cannibal = percent_cannibal / n
    return [n, m, kmean, kmax, C, node_connectivity, MGD, FFL_count, FBL_count, specialized_nodes, percent_cannibal]

In [28]:
def get_temp(filename):
    # returns temperature as a float
    df = pd.read_csv("our_references.csv")
    temp = df[["Filename", "Temperature"]]
    temperature = temp.loc[temp["Filename"] == filename,"Temperature"].iloc[0]
    return float(temperature)

In [29]:
def get_correlation_coefficients(data_array):
    # returns correlation coefficients as a list
    df = pd.DataFrame(data_array, columns = ['network', "nodes", "degrees", 'average degrees', 'highest degree', 'Cluster Coeff', 'connectivity', 'MGD', 
                                             'FFL', 'FBL', 'special nodes', 'percent_cannibal', 'Temperature'])
    correlations = df.corr()['Temperature'][:-1]
    print(df)
    return correlations #this line for returning as dataframe
    #return correlations.to_numpy()

In [30]:
# iterates through networks directory and reads each in as networkx graph G
directory = 'networks'
data = []
show = True
for filename in os.scandir(directory):
    filename = filename.name
    name, ext = os.path.splitext(filename)
    ext = str(ext)[0:4]
    if ext == '.txt' and filename != 'README':
        G = nx.read_edgelist(directory + '/' + filename, create_using=nx.DiGraph)
        network_stats = [filename]
        network_stats = network_stats + gather_network_stats(G)
        network_stats.append(get_temp(filename[:-4]))
        data.append(network_stats)
        if filename == "FW_003.txt" or filename == "FW_006.txt":
            species = pd.read_csv("species/"+ filename[:-4] + ".csv").columns[1:]
            mapping = {}
            for i, a in enumerate(G.nodes()):
                mapping[a] = species[int(a)]
#                 print(len(G.nodes()))
#                 print(len(species))
                a = species[int(a)]
                #print(a)
            H = nx.relabel_nodes(G, mapping)
            stats = gather_network_stats(G)
            #nx.draw(H, with_labels = True)
            net = Network(notebook = True)
            net = Network('2000px', '2000px', directed = True)
            net.from_nx(H)
            labels = ["nodes", "number of edges", 'average degrees', 'highest degree', 'Cluster Coeff', 'connectivity', 'MGD', 
                                             'FFL', 'FBL', 'special nodes', 'percent cannibal']
            for i, a in enumerate(stats):
                print(labels[i], ': ', a)
            for edge in net.edges:
                edge['label'] = ''
            net.show("test" + filename[3:-4] + ".html")
            show = False

nodes :  26
number of edges :  125
average degrees :  9.615384615384615
highest degree :  20
Cluster Coeff :  0.3758503401360544
connectivity :  0
MGD :  2.401666666666667
FFL :  9.923076923076923
FBL :  2.0384615384615383
special nodes :  0.11538461538461539
percent cannibal :  0.11538461538461539
nodes :  30
number of edges :  225
average degrees :  15.0
highest degree :  32
Cluster Coeff :  0.26447016918967053
connectivity :  0
MGD :  1.9350132625994696
FFL :  23.733333333333334
FBL :  3.3333333333333335
special nodes :  0.03333333333333333
percent cannibal :  0.2


In [31]:
get_correlation_coefficients(data)

          network  nodes  degrees  average degrees  highest degree  \
0      FW_001.txt     39      183         9.384615              26   
1      FW_002.txt     11       36         6.545455              11   
2      FW_003.txt     26      125         9.615385              20   
3      FW_004.txt     32      137         8.562500              19   
4      FW_005.txt     43      409        19.023256              54   
5      FW_006.txt     30      225        15.000000              32   
6      FW_007.txt     42      219        10.428571              20   
7      FW_008.txt    247     3301        26.728745             199   
8      FW_009.txt     37      237        12.810811              34   
9      FW_010.txt     35      247        14.114286              42   
10     FW_011.txt     71      338         9.521127              50   
11  FW_012_01.txt     37      120         6.486486              26   
12  FW_012_02.txt     44      145         6.590909              32   
13  FW_013_01.txt   

nodes               0.357052
degrees             0.209314
average degrees    -0.159898
highest degree      0.391070
Cluster Coeff       0.212016
connectivity        0.267024
MGD                -0.277938
FFL                -0.297519
FBL                -0.254127
special nodes       0.029932
percent_cannibal   -0.171546
Name: Temperature, dtype: float64

In [47]:
df = pd.DataFrame(data, columns = ['network', "nodes", "degrees", 'average degrees', 'highest degree', 'Cluster Coeff', 'connectivity', 'MGD',
                                   'FFL', 'FBL', 'special nodes', 'percent cannibal', 'Temperature'])
X = df[["nodes", "degrees", 'average degrees', 'highest degree', 'Cluster Coeff', 'connectivity', 
        'MGD', 'FFL', 'FBL', 'special nodes', 'percent cannibal']]
y = df['Temperature']

num_trials = 10000
rand_set = list(range(0, 33))
rand_set_track = list(range(0, 33))
average_errors = []
X = X.to_numpy()
y = y.to_numpy()
empirical_net = {}
for trial in range(0, num_trials):
    # create lists and generate random indeces for training from global dataset
    errors = []
    empirical_set = []
    training_set = []
    empirical_set_temps = []
    training_set_temps = []
    training_set_indeces = np.random.choice(rand_set, size = 30, replace=False)
    rand_set_track = [val for val in rand_set if val not in training_set_indeces]
    #print(rand_set_track)
    # append generated values to respective sets
    for index in training_set_indeces:
        training_set.append(X[index])
        training_set_temps.append(y[index])
    for index in rand_set_track:
        empirical_set.append(X[index])
        empirical_set_temps.append(y[index])
    # machine learning regression tree
    ML = tree.DecisionTreeRegressor()
    ML.fit(training_set, training_set_temps)
    for index in range(0, len(empirical_set_temps)):
        prediction = ML.predict([empirical_set[index]])
        errors.append(abs(empirical_set_temps[index] - prediction))
        if rand_set_track[index] in empirical_net:
            empirical_net[rand_set_track[index]].append((empirical_set_temps[index] - prediction)[0])
        else:
            #print("here")
            empirical_net[rand_set_track[index]] = (empirical_set_temps[index] - prediction).tolist()
            #print(empirical_net)
    average_errors.append(np.mean(errors))
for i in empirical_net:
    empirical_net[i] = np.mean(empirical_net[i])
empirical_net
print("Mean error across all runs:", np.mean(average_errors))

Mean error across all runs: 5.691696666666666
